In [ ]:
## VISSIM Modules
import win32com.client as com
import os

## RL Modules
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("ERROR: GPU DEVICE NOT FOUND.")

from keras.models import load_model
    
## Data Management Modules
import pickle

## User Defined Modules
import Simulator_Functions as SF

from RLAgents import DQNAgent
from NParser import NetworkParser
from COMServer import COMServerDispatch, COMServerReload
from TupleToList import toList
## Other Modules
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Network Model Parameters
Random_Seed = 42
model_name  = 'Single_Cross_Straight'
vissim_working_directory = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Vissim\\'
program = 'DuelingDQN' # DQN, DuelingDQN, DDQN, DuelingDDQN
reward_type = 'Queues'
state_type  = 'Queues'
## Use of additional files?
flag_read_additionally  = False
## Load trained model?
Demo_Mode = False
load_trained = False
Quickmode = True
SaveResultsAgent = True
# Random demand
Random_Demand = False

In [ ]:
## Data handling flags
# Flag for restarting the COM Server
reset_flag = True
#cache_flag = False
# If a fresh start is needed, all previous results from simulations are deleted
Start_Fresh = True
# Debug action
debug_action = False

In [ ]:
## RL Hyperparamenters
# Number of simulations, save every "n" episodes and copy weights with frequency "f"
episodes = 400
partial_save_at = 10
copy_weights_frequency = 5
reset_frequency = 10
# Timesteps per simulation (1 timestep = 0.1 sec), length for random population is a multiple of episode
simulation_length = 36000*1
memory_population_length = simulation_length*5
## State-Action Parameters
state_size = 4
action_size = 5
# Memory Size
memory_size = 5000
# Batch Size
batch_size = 128
# Learning Rate
alpha   = 0.0001
# Discount Factor
gamma   = 0.9
# Exploration Schedule
epsilon_start = 1
epsilon_end   = 0.001
epsilon_decay = (epsilon_end - epsilon_start)/(episodes-1)
#epsilon_decay = np.power(epsilon_end/epsilon_start, 1./(episodes+1)) # Geometric decay
# Demand Schedule
demands = [100, 200, 400, 600, 800, 1000]
# Session ID
Session_ID = 'Episodes'+str(episodes)+'_Program'+program

In [ ]:
if __name__ == "__main__":
    # Initialize storage
    reward_storage = []
    best_agent_weights = []
    loss=[]
    
    # If previous agents aren't to be loaded, proceed to memory population
    if not load_trained:
        # Initialize simulation
        if 'Vissim' not in globals() or Vissim == None:
            Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                                        memory_population_length, Start_Fresh,\
                                                                        reset_flag = True, verbose = True)
        else:
            Vissim = com.Dispatch("Vissim.Vissim")
            Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                                  memory_population_length, Start_Fresh, reset_flag = True)
        
    # Setting Random Seed
    Vissim.Simulation.SetAttValue('RandSeed', Random_Seed)
    print ('Random seed set in simulator. Random Seed = '+str(Random_Seed))

    # Deploy Network Parser (crawl network)
    npa = NetworkParser(Vissim)
    print('NetworkParser has succesfully crawled the model network.')
    
    # Initialize agents
    if program == "DQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = False, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DuelingDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = False, Dueling = True) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = True, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DuelingDDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = True, Dueling = True) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    else:
        print("Incorrect Agent Class selected. Deployment could not be completed.")
        quit()
    
    if agents_deployed:
        print("Deployed {} agent(s) of the Class {}.".format(len(Agents), program))
    
    if Demo_Mode:
        Agents, loss = SF.load_agents(vissim_working_directory, model_name, Agents, Session_ID, loss, best = True)
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, memory_population_length, Demo_Mode)
        Vissim = None
    # Load previous trained data
    elif load_trained:
        Agents, loss = SF.load_agents(vissim_working_directory, model_name, Agents, Session_ID, loss, best = False)
    # If previous data isn't to be loaded, have an initial longer random run to populate memory
    else:
        print('Populating memory with Random Actions....')
        SF.Set_Quickmode(Vissim)
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, memory_population_length, Demo_Mode)
    
    # Iterations of the simulation
    for episode in range(episodes):
        # Completely re-dispatch server every N iterations for performance
        if episode % reset_frequency == 0 and episode !=0:
            Vissim = None
            Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                                        simulation_length, Start_Fresh,\
                                                                        reset_flag = True, verbose = False)
            print("Redispatched")
        else:
            # If not the first episode, reset state at the start
            Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                                simulation_length, Start_Fresh, reset_flag = True)
        npa = NetworkParser(Vissim) 
        for index, agent in enumerate(Agents):
            agent.update_IDS(npa.signal_controllers_ids[index], npa)
            agent.episode_reward = []
        
        # Change demand for every episode
        if Random_Demand:
            for vehicle_input in range(1,5):
                Vissim.Net.VehicleInputs.ItemByKey(vehicle_input).SetAttValue('Volume(1)', demands[np.random.randint(0,len(demands)-1)])    
        
        # Run Episode at maximum speed

        SF.Set_Quickmode(Vissim)
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, simulation_length, Demo_Mode)
        
        # Calculate episode average reward
        reward_storage, average_reward = SF.average_reward(reward_storage, Agents, episode, episodes)
        best_agent_weights = SF.best_agent(reward_storage, average_reward, best_agent_weights, vissim_working_directory, model_name, Agents, Session_ID)
        
        # Train agent with experience of episode (indicated batch size)
        for agent in Agents:
            loss = agent.replay(batch_size, episode, loss)
        # Security save for long trainings
        if SaveResultsAgent:
            if (episode+1)%partial_save_at == 0:
                SF.save_agents(vissim_working_directory, model_name, Agents, Session_ID, reward_storage, loss)
                print('Saved Partial results at the end of episode {}.'.format(episode+1))

    #Saving agents memory, weights and optimizer
    if SaveResultsAgent:
        SF.save_agents(vissim_working_directory, model_name, Agents, Session_ID, reward_storage, loss)
        print("Model, architecture, weights, optimizer, memory and training results succesfully saved. Succesfully Terminated.")
    
    # Plotting training progress
    plt.figure(figsize=(8,4.5))
    x_series = range(1,len(reward_storage)+1)
    fit = np.polyfit(x_series,reward_storage,1)
    fit_fn = np.poly1d(fit) 
    plt.plot(x_series,reward_storage, '-b', x_series, fit_fn(x_series), '--r')
    plt.xlabel('Episodes')
    plt.ylabel('Average ageng reward in episode')
    plt.title('Training evolution and trend')
    plt.gca().legend(('Episode Reward','Linear Trend'))
    plt.show()
    
    # Plotting training loss
    plt.figure(figsize=(8,4.5))
    x_series = range(1,len(loss)+1)
    plt.plot(x_series,loss, '-b')
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.title('Model Loss')
    plt.gca().legend(('Loss'))
    plt.show()
    
    # Close Vissim
    Vissim = None

In [ ]:
Demo_Mode = True
if __name__ == "__main__":
    # Initialize storage
    reward_storage = []
    
    # If previous agents aren't to be loaded, proceed to memory population
    if not load_trained:
        # Initialize simulation
        if 'Vissim' not in globals() or Vissim == None:
            Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                                        memory_population_length, Start_Fresh, reset_flag = True)
        else:
            Vissim = com.Dispatch("Vissim.Vissim")
            Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                                  memory_population_length, Start_Fresh, reset_flag = True)
        
    # Setting Random Seed
    Vissim.Simulation.SetAttValue('RandSeed', Random_Seed)
    print ('Random seed set in simulator. Random Seed = '+str(Random_Seed))

    # Deploy Network Parser (crawl network)
    npa = NetworkParser(Vissim)
    print('NetworkParser has succesfully crawled the model network.')
    
    # Initialize agents
    if program == "DQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, Vissim, DoubleDQN = False, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, Vissim, DoubleDQN = True, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DuelingDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = True, Dueling = True) for ID in npa.signal_controllers_ids] 
      
        agents_deployed = True
    else:
        print("Incorrect Agent Class selected. Deployment could not be completed.")
        quit()
    
    if agents_deployed:
        print("Deployed {} agent(s) of the Class {}.".format(len(Agents), program))
    
    if Demo_Mode:
        for index, agent in enumerate(Agents):
            Filename = os.path.join(vissim_working_directory, model_name, model_name+'_'+ Session_ID + '_Agent'+str(index)+'.h5')
            agent.model = load_model(Filename)
            Memory_Filename = os.path.join(vissim_working_directory, model_name, model_name+'_'+ Session_ID + '_Agent'+str(index)+'_Memory'+'.p')
            agent.memory = pickle.load(open(Memory_Filename, 'rb'))
        print('Items successfully loaded.')        
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, memory_population_length, Demo_Mode = True)
        Vissim = None

In [ ]:
 # Plotting training progress
    plt.figure(figsize=(8,4.5))
    x_series = range(1,len(reward_storage)+1)
    fit = np.polyfit(x_series,reward_storage,1)
    fit_fn = np.poly1d(fit) 
    plt.plot(x_series,reward_storage, '-b', x_series, fit_fn(x_series), '--r')
    plt.xlabel('Episodes')
    plt.ylabel('Average ageng reward in episode')
    plt.title('Training evolution and trend')
    plt.gca().legend(('Episode Reward','Linear Trend'))
    plt.show()
    

In [ ]:
# Plotting training loss
bo = loss[0:128*10]
x = np.arange(0, len(bo), 128)
plt.figure(figsize=(16,9))
x_series = range(1,len(bo)+1)
plt.plot(x_series,np.sqrt(bo), '-b')
for xc in x:
    plt.axvline(xc)
plt.xlabel('Training Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.gca().legend(('Loss'))
plt.show()

In [ ]:
Training_Progress_Filename = os.path.join(vissim_working_directory, model_name, model_name+'_'+ Session_ID + '_Agent'+str(index)+'_Training'+'.p')
pickle.dump(reward_storage, open(Training_Progress_Filename, 'wb'))


In [ ]:
Agents[0].epsilon